# GPS Traces to Places

Transform raw GPS data into meaningful places with familiarity.

## Pipeline
1. Load GPX trace → individual points
2. Cluster points into spatial extents
3. Create encounters for each visit
4. Derive familiarity over time
5. Extract emergent places

In [ ]:
from chora.core import PlatialGraph, Agent, SpatialExtent, Encounter, PlatialEdge
from datetime import datetime, timedelta
import random

graph = PlatialGraph(name="gps_demo")
agent = Agent.individual("walker")
graph.add_node(agent)

## Simulating GPS Data

We'll simulate a week of GPS traces visiting several locations.

In [ ]:
# Define some places the person visits regularly
locations = [
    {"name": "Home", "lon": -0.090, "lat": 51.510, "frequency": 0.4},
    {"name": "Office", "lon": -0.100, "lat": 51.520, "frequency": 0.3},
    {"name": "Gym", "lon": -0.085, "lat": 51.515, "frequency": 0.15},
    {"name": "Cafe", "lon": -0.095, "lat": 51.512, "frequency": 0.1},
    {"name": "Park", "lon": -0.110, "lat": 51.525, "frequency": 0.05},
]

# Create spatial extents for each location
extents = {}
for loc in locations:
    ext = SpatialExtent.from_point(lon=loc["lon"], lat=loc["lat"], name=loc["name"])
    graph.add_node(ext)
    extents[loc["name"]] = ext
    
print(f"Created {len(extents)} places")

In [ ]:
# Simulate 2 weeks of visits
num_days = 14
visits_per_day = 4

all_encounters = []
for day in range(num_days):
    for visit in range(visits_per_day):
        # Pick a location based on frequency weights
        rand = random.random()
        cumulative = 0
        selected = locations[0]
        for loc in locations:
            cumulative += loc["frequency"]
            if rand <= cumulative:
                selected = loc
                break
        
        # Create encounter
        enc = Encounter(
            agent_id=agent.id,
            extent_id=extents[selected["name"]].id,
            start_time=datetime.now() - timedelta(days=num_days-day, hours=8+visit*4),
            activity="visit"
        )
        graph.add_node(enc)
        graph.add_edge(PlatialEdge.participates_in(agent.id, enc.id))
        graph.add_edge(PlatialEdge.occurs_at(enc.id, extents[selected["name"]].id))
        all_encounters.append(enc)

print(f"Created {len(all_encounters)} encounters over {num_days} days")

## Deriving Familiarity

In [ ]:
from chora.derive import update_familiarity

# Process encounters chronologically
sorted_encounters = sorted(all_encounters, key=lambda e: e.start_time)

for enc in sorted_encounters:
    update_familiarity(graph, enc)
    
print("Familiarity scores updated!")

## Extracting Emergent Places

In [ ]:
from chora.derive.place import extract_place

# Extract emergent places for each spatial extent
for name, ext in extents.items():
    place = extract_place(graph, ext.id, agent.id)
    
    print(f"\n{name}:")
    print(f"  Familiarity: {place.familiarity_score:.3f}")
    print(f"  Encounter count: {place.encounter_count}")

## Query Familiar Places

In [ ]:
from chora.query import find_familiar_places

familiar = find_familiar_places(graph, agent.id, min_familiarity=0.3)

print(f"\nPlaces with familiarity >= 0.3:")
for p in sorted(familiar, key=lambda x: x.familiarity_score, reverse=True):
    print(f"  {p.extent.name}: {p.familiarity_score:.3f}")